In [270]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat
%matplotlib inline

In [3]:
data = loadmat('data/ex3data1.mat')
data               

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
X = data['X'] 
y = data['y']

X.shape, y.shape

((5000, 400), (5000, 1))

### Logistic Regression (One-vs-all)

In [244]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [245]:
def logistic_cost(theta, X, y, lamda):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    inner = - np.multiply(y, np.log(sigmoid(X * theta.T))) - (np.multiply((1-y), np.log(1 - sigmoid(X * theta.T))))
    reg_term = (lamda /(2 * len(X))) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return (np.sum(inner) / len(X)) + reg_term

In [246]:
def logistic_GD(theta, X, y, lamda):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    GD = (((sigmoid(X * theta.T) - y).T * X) / (len(X))) + ((lamda/len(X))*theta)
#    GD[0,0] = GD[0,0] - (lamda/len(X))*theta.T[0,0]
    GD[0, 0] = (np.sum(np.multiply((sigmoid(X * theta.T) - y), X[:,0]))) / len(X)
    return np.array(GD).ravel()

In [117]:
alpha = 0.01
iters = 1000
# GD estimates
def GD_estimates(X, y, theta, alpha, iters, lamda):
    cost = []
    for _ in range(iters):
        theta = logistic_GD (X, y, theta, alpha, lamda)
        cost.append(logistic_cost(X, y, theta))
    cost = np.array(cost)
    return theta, cost  

In [118]:
type(data['X'])

numpy.ndarray

In [186]:
X = data['X']
rows = X.shape[0]
X = np.insert(X, 0, values=np.ones(rows), axis=1)
X

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.]])

In [247]:
from scipy.optimize import minimize

def One_vs_all(X, y, num_labels, lamda):  
    rows = X.shape[0]
    params = X.shape[1]

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=logistic_cost, x0=theta, args=(X, y_i, lamda), method='TNC', jac=logistic_GD)
        all_theta[i-1,:] = fmin.x

    return all_theta

In [121]:
y_0 = np.array([1 if label == 0 else 0 for label in data['y']]) 
#y_0 = np.reshape(y_0, (rows, 1))
type(y_0)

numpy.ndarray

In [122]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [148]:
rows = data['X'].shape[0]  
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)
#theta = np.reshape(theta, (1, (params + 1))) 
theta = np.matrix(theta)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])  
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (1, 401), (10, 401))

In [248]:
all_theta = One_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[ -2.38291346e+00,   0.00000000e+00,   0.00000000e+00, ...,
          1.30474743e-03,  -8.24531507e-10,   0.00000000e+00],
       [ -3.18403111e+00,   0.00000000e+00,   0.00000000e+00, ...,
          4.46025411e-03,  -5.08531235e-04,   0.00000000e+00],
       [ -4.80012163e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -2.88116417e-05,  -2.47992614e-07,   0.00000000e+00],
       ..., 
       [ -7.98722644e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -8.95295280e-05,   7.21832616e-06,   0.00000000e+00],
       [ -4.57235036e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.33349450e-03,   9.96261157e-05,   0.00000000e+00],
       [ -5.40509120e+00,   0.00000000e+00,   0.00000000e+00, ...,
         -1.16616386e-04,   7.88424336e-06,   0.00000000e+00]])

In [249]:
def predict_all(X, all_theta):  
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [250]:
y_pred = predict_all(data['X'], all_theta)  
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
#print 'accuracy = {0}%'.format(accuracy * 100)
accuracy

0.9446

In [205]:
all_theta.shape

(10, 401)

In [221]:
all_theta = np.matrix(all_theta)
h = sigmoid(X * all_theta.T)
h_argmax = np.argmax(h, axis=1)
h_argmax

matrix([[9],
        [9],
        [9],
        ..., 
        [8],
        [8],
        [6]], dtype=int64)

### Neural Networks

In [41]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)

y[1,:], y_onehot[1,:]


(matrix([[10]], dtype=uint8),
 array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.]))

In [42]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [43]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)
    
    return a1, z2, a2, z3, h

In [51]:
def cost(params, input_size, hidden_size, num_labels, X, y, lamda):
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)
    
    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    
    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    
    # compute the cost
    J = 0
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / m
    J += (float(lamda) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))  
    return J

In [117]:
# initial setup
input_size = 400  
hidden_size = 25  
num_labels = 10  
lamda = 1

# randomly initialize a parameter array of the size of the full network's parameters
params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]  
X = np.matrix(X)  
y = np.matrix(y)

# unravel the parameter array into parameter matrices for each layer
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [118]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)  
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [119]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, lamda)

7.040207443328292

In [120]:
def sigmoid_gradient(z):  
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

In [121]:
def back_propagate(params, input_size, hidden_size, num_labels, X, y, lamda):  
    ##### this section is identical to the cost function logic we already saw #####
    m = X.shape[0]
    X = np.matrix(X)
    y = np.matrix(y)

    # reshape the parameter array into parameter matrices for each layer
    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

    # run the feed-forward pass
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    # initializations
    J = 0
    delta1 = np.zeros(theta1.shape)  # (25, 401)
    delta2 = np.zeros(theta2.shape)  # (10, 26)

    # compute the cost
    for i in range(m):
        first_term = np.multiply(-y[i,:], np.log(h[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - h[i,:]))
        J += np.sum(first_term - second_term)

    J = J / m

    # add the cost regularization term
    J += (float(lamda) / (2 * m)) * (np.sum(np.power(theta1[:,1:], 2)) + np.sum(np.power(theta2[:,1:], 2)))

    ##### end of cost function logic, below is the new part #####

    # perform backpropagation
    for t in range(m):
        a1t = a1[t,:]  # (1, 401)
        z2t = z2[t,:]  # (1, 25)
        a2t = a2[t,:]  # (1, 26)
        ht = h[t,:]  # (1, 10)
        yt = y[t,:]  # (1, 10)

        d3t = ht - yt  # (1, 10)

        z2t = np.insert(z2t, 0, values=np.ones(1))  # (1, 26)
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))  # (1, 26)
#        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(a2t))  # (1, 26)
        
        delta1 = delta1 + (d2t[:,1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    delta1 = delta1 / m
    delta2 = delta2 / m

    # add the gradient regularization term
    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * lamda) / m
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * lamda) / m

    # unravel the gradient matrices into a single array
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [122]:
J, grad = back_propagate(params, input_size, hidden_size, num_labels, X, y_onehot, learning_rate)  
J, grad.shape

(7.040207443328292, (10285,))

In [123]:
from scipy.optimize import minimize

# minimize the objective function
fmin = minimize(fun=back_propagate, x0=params, args=(input_size, hidden_size, num_labels, X, y_onehot, lamda),  
                method='TNC', jac=True, options={'maxiter': 250})
fmin  

     fun: 0.34134679008640101
     jac: array([  6.60507811e-04,   1.33900129e-06,   1.01261993e-06, ...,
        -5.14748824e-04,  -1.03455682e-04,  -4.30096287e-04])
 message: 'Max. number of function evaluations reached'
    nfev: 250
     nit: 20
  status: 3
 success: False
       x: array([-1.1114729 ,  0.00669501,  0.0050631 , ...,  0.28414764,
       -1.28418555,  0.7518175 ])

In [124]:
X = np.matrix(X)  
theta1 = np.matrix(np.reshape(fmin.x[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))  
theta2 = np.matrix(np.reshape(fmin.x[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)  
y_pred = np.array(np.argmax(h, axis=1) + 1)  
y_pred 

array([[10],
       [10],
       [10],
       ..., 
       [ 9],
       [ 9],
       [ 9]], dtype=int64)

In [125]:
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]  
accuracy = (sum(map(int, correct)) / float(len(correct)))
#accuracy = sum(correct) / len(correct)
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 99.3%
